# Passage en donnée catégorielle pour la détection de fréquences récurrentes

### Import libraries

In [1]:
import pandas as pd
import numpy as np

### Import datasets

In [3]:
data =  pd.read_csv("../data_pretraitee/data_pretraite.csv", sep=';', index_col = 0, parse_dates=['Date'])
meteo = pd.read_excel("../data/meteo_2017_2022.xlsx")

In [4]:
meteo_18_21=meteo[(meteo["Année"]!=2017) & (meteo["Année"]!=2022)]

In [6]:
data.columns

Index(['N2O', 'Date', 'Bande', 'soilH_h1', 'soilH_h2', 'NO3_h1', 'NO3_h2',
       'NH4_h1', 'NH4_h2', 'meanTP', 'air_humidity', 'pluie',
       'Travail_sol_J90', 'Semis_J90', 'CP', 'Peuplement', 'fertilisation',
       'nbJours_ferti', 'Ajout_phyto_30J', 'Ajout_phyto_45J',
       'Ajout_phyto_60J', 'Ajout_ferti_30J', 'Ajout_ferti_45J',
       'Ajout_ferti_60J', 'Recolte', 'mean_soilH_h1_J5', 'mean_soilH_h2_J5',
       'mean_NH4_h1_J5', 'mean_NH4_h2_J5', 'mean_NO3_h1_J5', 'mean_NO3_h2_J5',
       'N2O_O3', 'N2O_J45', 'N2O_J182', 'Humidite_max_5J', 'Humidite_min_5J',
       'Humidite_moyenne_5J', 'Temp_max_5J', 'Temp_min_5J', 'Temp_moyenne_5J',
       'Sum_ETP_5J', 'Sum_pluie_5J', 'acs', 'pn', 'bi', 'bio', 'elev', 'ref',
       'week', 'month', 'year', '2018', '2019', '2020', '2021', 'Pic_03',
       'Pic_J45', 'Pic_J182'],
      dtype='object')

In [5]:
data2 = data[["N2O", "Date", "Bande", "CP", "Peuplement"
            , "Travail_sol_J90", "Ajout_phyto_30J", "Ajout_ferti_30J"
            , "N2O_O3", "N2O_J45", "N2O_J182", "Temp_moyenne_5J"
            , "Sum_pluie_5J", "year", "month"]]

### Define Class Categorizer

In [5]:
class categorizer :
    """provide functions to transform a quantitative column to a qualitative column"""

    def __init__(self, df_cible: object) -> None:
        self.df = df_cible

    def by_quantile(self, df_quantile: object, col_quantile : str, col_cible : str, name_col : str, factor : int = 1) -> object: 
        """
        df_quantile : Dataframe we will used for quantile
        col_quantile : The column that will be used to estimate the quantiles [0.25, 0.75]
        factor : Possibility to multiply the quantiles by a factor (usefull if we compare a daily value to a weekly value)
        col_cible : The column that wil be categorized according to the quantiles found
        Name_col : Name of the new column
        """
        quantiles = df_quantile[col_quantile].quantile([0.25, 0.75])
        self.df[name_col] = np.select(
                            [self.df[col_cible] >= (quantiles[0.75] * factor)
                            , self.df[col_cible] <= (quantiles[0.25] * factor)
                            , (self.df[col_cible] > (quantiles[0.25] * factor)) & (self.df[col_cible] < (quantiles[0.75] * factor))]
                            , ["elevee","basse","moyenne"])
        return (self.df)

    def by_values(self, col_cible : str, values_int : list, values_str : list, name_col : str) -> object :
        """
        col_cible : The column that wil be categorized according to the values_int
        values_int : List of int used to separate the col_cible
        values_str : Names of the categories
        Name_col : Name of the new column
        """
        for i in range(len(values_int)):
            # print(values_int[i - 1], values_int[i], values_str[i])
            self.df.loc[(self.df[col_cible] > values_int[i - 1])
                        & (self.df[col_cible] <= values_int[i])
                        , name_col] = values_str[i]
        return self.df

    def compare_cols(self, df_source : object, col_cible : str, col_source : str, col_name : str) -> object :
        """
        df_source : Dataframe use for comparing
        col_cible : Name of the column on df_source
        col_source : Name of the column we are looking to categorized
        col_name : Name of the new column
        """
        self.df[col_name] = self.df[col_cible] > df_source[col_source]
        return self.df

### Data Transformation

In [6]:
cat = categorizer(data2.copy())

data2 = cat.by_quantile(meteo_18_21, "Temp moyenne (°C)", "Temp_moyenne_5J", "Temp_quali")
data2 = cat.by_quantile(meteo_18_21, "Pluie (mm)", "Sum_pluie_5J", "Pluie_quali", 5)
data2 = cat.by_values("Sum_pluie_5J", [1, data2.Sum_pluie_5J.max(),0], ['pas_pluie', 'pluie', 'pas_pluie'], 'Pluie_ON')
data2["day"] = data2.Date.dt.day_of_year
data2 = cat.by_values("day", [80, 172, 264, 355, 0], ['hiver', 'printemps', 'ete', 'automne','hiver'], "Saison")
data2.pop("day")
data2 = cat.compare_cols(data2, "N2O", "N2O_O3", "Pic_O3")
data2 = cat.compare_cols(data2, "N2O", "N2O_J45", "Pic_J45")
data2 = cat.compare_cols(data2, "N2O", "N2O_J182", "Pic_J182")

In [7]:
data2.head()

,N2O,Date,Bande,CP,Peuplement,Travail_sol_J90,Ajout_phyto_30J,Ajout_ferti_30J,N2O_O3,N2O_J45,...,Sum_pluie_5J,year,month,Temp_quali,Pluie_quali,Pluie_ON,Saison,Pic_O3,Pic_J45,Pic_J182
0,18.537129,2018-03-14,acs,ble,ble,False,0.0,1.0,11.582333,24.452717,...,28.0,2018,3,moyenne,elevee,pluie,hiver,True,False,True
1,2.088998,2018-03-14,pn,ble,ble,False,0.0,1.0,2.082846,8.071064,...,28.0,2018,3,moyenne,elevee,pluie,hiver,True,False,False
2,1.488817,2018-03-14,bi,ble,ble,False,0.0,0.0,1.166688,6.928661,...,28.0,2018,3,moyenne,elevee,pluie,hiver,True,False,False
3,1.390654,2018-03-14,bio,ble,ble,False,0.0,1.0,2.004849,8.051543,...,28.0,2018,3,moyenne,elevee,pluie,hiver,False,False,False
4,3.297613,2018-03-14,elev,ble,ble,False,0.0,0.0,2.746295,14.524024,...,28.0,2018,3,moyenne,elevee,pluie,hiver,True,False,False


In [8]:
# data3 = data2.iloc[:,15:]

### Export data

In [9]:
data2.to_csv("../data_pretraitee/data_categorielle.csv", sep=";")